In [1]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

In [2]:
mnist = read_data_sets('data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

In [4]:
def sigmoid(x):
    sigmoidX = tf.div(tf.constant(1.0), #1.0/(1+e^-x)
                   tf.add ( tf.constant(1.0), tf.exp(tf.negative(x)) )
                   )
    return sigmoidX

In [5]:
def softmax(x):
    X = tf.exp(x) / tf.reduce_sum(tf.exp(x), axis=0)
    return X

In [6]:
W1 = tf.Variable(tf.random_normal([784, 200], stddev=0.1))
b1 = tf.Variable(tf.random_normal([200], stddev=0.1))
z1 = tf.add(tf.matmul(X, W1), b1) #layer1 logit
L1 = sigmoid(tf.add(tf.matmul(X, W1), b1)) #sigmoid, layer1

W2 = tf.Variable(tf.random_normal([200, 100], stddev=0.1))
b2 = tf.Variable(tf.random_normal([100], stddev=0.1))
z2 = tf.add(tf.matmul(L1, W2), b2) #layer logit2
L2 = sigmoid(tf.add(tf.matmul(L1, W2), b2)) #layer2

W3 = tf.Variable(tf.random_normal([100, 10], stddev=0.1))
b3 = tf.Variable(tf.random_normal([10], stddev=0.1))
logits = tf.add(tf.matmul(L2, W3), b3)# output logit
H = softmax(logits)# Hypothesis
#순전파

In [7]:
def sigmoidprime(x):
    sigmoidPX = tf.multiply(sigmoid(x), 
                          tf.subtract( tf.constant(1.0), sigmoid(x) )
                         )
    return sigmoidPX

In [8]:
diff = tf.subtract(H, Y)#예측값-실제값(오차)

In [9]:
d_z3 = tf.multiply(diff, sigmoidprime(logits))#오차 x logits, 결과값에 로짓이 미치는 영향, 체인룰
d_b3 = d_z3
d_W3 = tf.matmul(tf.transpose(L2), d_z3)

In [10]:
d_L2 = tf.matmul(d_z3, tf.transpose(W3))
d_z2 = tf.multiply(d_L2, sigmoidprime(z2))
d_b2 = d_z2
d_W2 = tf.matmul(tf.transpose(L1), d_z2)

In [11]:
d_L1 = tf.matmul(d_z2, tf.transpose(W2))
d_z1 = tf.multiply(d_L1, sigmoidprime(z1))
d_b1 = d_z1
d_W1 = tf.matmul(tf.transpose(X), d_z1)
#역전파

In [12]:
eta = tf.constant(0.5)

In [13]:
step = [
    tf.assign(W1, tf.subtract(W1, tf.multiply(eta, d_W1)))
  , tf.assign(b1, tf.subtract(b1, tf.multiply(eta, tf.reduce_mean(d_b1, axis=0))))
    
  , tf.assign(W2, tf.subtract(W2, tf.multiply(eta, d_W2)))
  , tf.assign(b2, tf.subtract(b2, tf.multiply(eta, tf.reduce_mean(d_b2, axis=0))))
    
  , tf.assign(W3, tf.subtract(W3, tf.multiply(eta, d_W3)))
  , tf.assign(b3, tf.subtract(b3, tf.multiply(eta, tf.reduce_mean(d_b3, axis=0))))
]

In [14]:
is_correct = tf.equal(tf.argmax(H, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(step, feed_dict = {X: batch_xs, Y : batch_ys})
    if i % 1000 == 0:
        a = sess.run(accuracy, feed_dict ={X: mnist.test.images,Y : mnist.test.labels})
        print(a)

0.148
0.789
0.8975
0.912
0.9242
0.9193
0.9279
0.9231
0.9154
0.9344


In [17]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(step, feed_dict = {X: batch_xs, Y : batch_ys})
    if i % 1000 == 0:
        h, a = sess.run([H, accuracy], feed_dict ={X: mnist.test.images,Y : mnist.test.labels})
        print(a,"\n", h)

0.9268 
 [[6.6048139e-18 1.3678983e-18 4.1708217e-22 ... 6.7068776e-03
  7.8645754e-25 2.8292538e-17]
 [6.4529824e-18 5.5403954e-18 1.6014503e-06 ... 8.7421250e-23
  7.9341832e-29 7.2837807e-24]
 [3.2411429e-23 3.6446413e-03 4.3676312e-21 ... 4.5392504e-15
  1.0278267e-22 1.6058665e-18]
 ...
 [2.8277021e-20 1.1707941e-17 1.5108005e-22 ... 2.2357242e-15
  5.5183357e-22 3.9278802e-15]
 [5.6317368e-16 2.0687039e-16 1.0648183e-24 ... 2.4420868e-18
  4.2537543e-19 1.1270572e-18]
 [1.2690600e-17 1.6462133e-17 6.8283698e-19 ... 3.2077776e-23
  1.8312691e-25 5.0897431e-19]]
0.9363 
 [[2.61298239e-19 9.39545344e-19 3.40064682e-18 ... 1.66737172e-03
  2.88410340e-27 2.92639337e-19]
 [7.08839936e-18 3.07088280e-18 4.77075490e-09 ... 3.26597313e-23
  5.55422247e-30 7.75566390e-24]
 [1.46976028e-25 7.72704429e-04 1.01577236e-19 ... 7.76145692e-15
  5.02517373e-24 3.33768734e-20]
 ...
 [6.42845958e-21 1.50053893e-18 3.25992138e-19 ... 6.02867918e-16
  1.21216195e-18 1.84318393e-15]
 [7.80016180e-18 